In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [5]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [6]:
json_Election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_Election_file.append(json.loads(line))

In [7]:
dict_train={}
i=0
for train_file in json_Election_file:
    dict_train[i]=train_file
    i+=1
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=False)
train.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,extended_tweet,possibly_sensitive,display_text_range,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,Fri Nov 20 20:44:34 +0000 2020,1329888388959309824,1329888388959309824,"RT @ElijahSchaffer: 7,778 people died on avera...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1605905074716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Nov 20 20:44:34 +0000 2020,1329888388854460421,1329888388854460421,"They knew by May, 2017 CIA #Scorecard had fail...","<a href=""http://twitter.com/download/iphone"" r...",True,1.329882e+18,1329881736193445895,7.887389e+17,788738939939090432,...,en,1605905074691,"{'full_text': 'They knew by May, 2017 CIA #Sco...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Nov 20 20:44:34 +0000 2020,1329888388992921603,1329888388992921603,RT @RepMattGaetz: We can win the future with P...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1605905074724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fri Nov 20 20:44:34 +0000 2020,1329888388929912834,1329888388929912834,Certified Surgical Technologist - CST - PRN - ...,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",True,NaN,None,NaN,None,...,en,1605905074709,{'full_text': 'Certified Surgical Technologist...,False,NaN,NaN,NaN,NaN,NaN,NaN
4,Fri Nov 20 20:44:34 +0000 2020,1329888388904837123,1329888388904837123,@Domenic_Toronto @RamonaFleury Ford is no long...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.329881e+18,1329880590867423233,1.638677e+09,1638677252,...,en,1605905074703,{'full_text': '@Domenic_Toronto @RamonaFleury ...,NaN,"[31, 140]",NaN,NaN,NaN,NaN,NaN
5,Fri Nov 20 20:44:34 +0000 2020,1329888389093613568,1329888389093613568,"RT @Annakhait: “Trump shall become a trumpet, ...","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1605905074748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fri Nov 20 20:44:34 +0000 2020,1329888389236019201,1329888389236019201,RT @KIRO7Seattle: Birthday time: Biden turns 7...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1605905074782,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
7,Fri Nov 20 20:44:34 +0000 2020,1329888389215236098,1329888389215236098,"RT @dandrezner: Now, it could be argued by som...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1605905074777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Fri Nov 20 20:44:34 +0000 2020,1329888389252993026,1329888389252993026,RT @thehill: President-elect Joe Biden arrives...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1605905074786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fri Nov 20 20:44:34 +0000 2020,1329888389315907586,1329888389315907586,RT @AOECOIN: #Breaking #BreakingNews \n💥🚨💥🚨\n#...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1605905074801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.set_option('display.max_columns', None)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 501 non-null    object 
 1   id                         501 non-null    int64  
 2   id_str                     501 non-null    object 
 3   text                       501 non-null    object 
 4   source                     501 non-null    object 
 5   truncated                  501 non-null    bool   
 6   in_reply_to_status_id      46 non-null     float64
 7   in_reply_to_status_id_str  46 non-null     object 
 8   in_reply_to_user_id        51 non-null     float64
 9   in_reply_to_user_id_str    51 non-null     object 
 10  in_reply_to_screen_name    51 non-null     object 
 11  user                       501 non-null    object 
 12  geo                        0 non-null      object 
 13  coordinates                0 non-null      object 

In [9]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet	| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)
def get_user_info(data: pd.DataFrame)->None:
    pass

In [10]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [11]:
for i in range(len(train)):
    train['text'][i]=text_normalization(train['text'][i])

train=remove_unnecessary_columns(train)

<ipython-input-11-dd037f2c4533>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'][i]=text_normalization(train['text'][i])


In [12]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=train['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

In [13]:
# coger screen_name, followers_count, friends_count, favorite_count
#  Tweet	| Username | Date | Hashtags | Likes | Retweets | Url

train.iloc[0]['extended_tweet']
#data['entities'][tweet]['urls']['expanded_tweet']
get_tweet_data(train)
#train

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,"7,778 people died on average every day in the ...",ELIJAH SCHAFFER,Fri Nov 20 16:52:56 +0000 2020,None,2602,677,https://twitter.com/i/web/status/1329888388959...
1,"They knew by May, 2017 CIA #Scorecard had fail...",Land of the Free,Fri Nov 20 20:44:34 +0000 2020,None,0,0,https://twitter.com/i/web/status/1329888388854...
2,We can win the future with President @realDona...,Rep. Matt Gaetz,Thu Nov 19 22:21:52 +0000 2020,None,4408,724,https://twitter.com/i/web/status/1329888388992...
3,Certified Surgical Technologist - CST - PRN - ...,DS Career Center,Fri Nov 20 20:44:34 +0000 2020,None,0,0,https://twitter.com/i/web/status/1329888388929...
4,@Domenic_Toronto @RamonaFleury Ford is no long...,HereOnEarth,Fri Nov 20 20:44:34 +0000 2020,None,0,0,https://twitter.com/i/web/status/1329888388904...
...,...,...,...,...,...,...,...
496,Lin Wood: “President Trump Won Over 400 Electo...,The DC Patriot,Fri Nov 20 15:40:41 +0000 2020,None,139,39,https://twitter.com/i/web/status/1329888428562...
497,crazi hometown wilmington delawar center unive...,Long Board Surfing Oil Tycoon Dude,Fri Nov 20 20:44:44 +0000 2020,None,0,0,https://twitter.com/i/web/status/1329888428532...
498,bplace01 100 lot damag done trump co way door,TacoTruck_on_sea🌮🔥🏦,Fri Nov 20 20:44:44 +0000 2020,None,0,0,https://twitter.com/i/web/status/1329888428826...
499,The Mockingbirds at FauxNews are getting more ...,Deplorable Smelly Hillbilly John,Fri Nov 20 15:34:12 +0000 2020,None,75,64,https://twitter.com/i/web/status/1329888428842...


In [14]:
os.remove(OUTPUT_FILENAME)

In [15]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN